In [ ]:
##Запуск PepairPDB

import subprocess
import os
import pandas as pd

# Путь до рабочей папки
wsl_path = ''

cmd = f"cd '{wsl_path}' && foldx --command=RepairPDB --pdb=SM_homology_AF_myh7_model_GN_HIS.pdb"
subprocess.run(["wsl", "bash", "-c", cmd], capture_output=True, text=True)

In [1]:
#Перевод списка мутации из формата ClinVar идентификатора в формат принимаемый FoldX-ом

variants = """
NM_000257.4(MYH7):c.5757G>C (p.Lys1919Asn)
"""
import re

mutant_list = re.findall(r'\(p\.([A-Za-z]+[0-9]+[A-Za-z]+)\)', variants)


aa_dict = {
    'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D',
    'Cys': 'C', 'Gln': 'Q', 'Glu': 'E', 'Gly': 'G',
    'His': 'H', 'Ile': 'I', 'Leu': 'L', 'Lys': 'K',
    'Met': 'M', 'Phe': 'F', 'Pro': 'P', 'Ser': 'S',
    'Thr': 'T', 'Trp': 'W', 'Tyr': 'Y', 'Val': 'V'
}

mutations = []

for i in mutant_list:
    for aa_3, aa_1 in aa_dict.items():
        i = i.replace(aa_3, aa_1)
    mutations.append(i)

print(mutations)

['K1919N']


In [2]:
# Формирование фацлов individual_list

import os

output_dir = r'' #Путь к папке для сохранения файлов 

########## Для цепи А

for mut in mutations:
    original_aa = mut[0]
    pos = mut[1:-1]
    new_aa = mut[-1]

    line = f"{original_aa}A{pos}{new_aa};\n"

    filename = f"individual_list_A_{mut}.txt"
    filepath = os.path.join(output_dir, filename)

    with open(filepath, 'w', encoding='ascii', newline='\n') as f:
        f.write(line)

########## Для цепи B
for mut in mutations:
    original_aa = mut[0]
    pos = mut[1:-1]
    new_aa = mut[-1]

    line = f"{original_aa}B{pos}{new_aa};\n"

    filename = f"individual_list_B_{mut}.txt"
    filepath = os.path.join(output_dir, filename)

    with open(filepath, 'w', encoding='ascii', newline='\n') as f:
        f.write(line)

In [3]:
import subprocess
import os
import pandas as pd

# Пути до рабочей папки
windows_path = r'' 
wsl_path = ''

mutation_files = [f for f in os.listdir(windows_path) if f.startswith("individual_list_") and f.endswith(".txt")]
results_dict = {}

for mut_file in mutation_files:
    parts = mut_file[:-4].split('_') 
    chain = parts[2] 
    mut_name = parts[3]  

    print(f"Запуск BuildModel для мутации {mut_name}, цепь {chain}")

    # PDB файл используемый в команде (SM_homology_AF_myh7_model_GN_HIS_Repair.pdb) лежит в репозитории
    cmd = f"cd '{wsl_path}' && foldx --command=BuildModel --pdb=SM_homology_AF_myh7_model_GN_HIS_Repair.pdb --mutant-file={mut_file}"
    
    result = subprocess.run(["wsl", "bash", "-c", cmd], capture_output=True, text=True)

    dif_file_path = os.path.join(windows_path, 'Dif_SM_homology_AF_myh7_model_GN_HIS_Repair.fxout')

    ddg = None
    if os.path.exists(dif_file_path):
        with open(dif_file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith('SM_homology_AF_myh7_model_GN_HIS_Repair_1.pdb'):
                    ddg = float(line.strip().split('\t')[1])
                    print(f"ddG для {mut_name}, цепь {chain}: {ddg}")
                    break
        os.remove(dif_file_path) #Для каждой новой мутации файл перезаписывается
    else:
        print(f"Результат не получен для мутации {mut_name}, цепь {chain}")
        
    if mut_name not in results_dict:
        results_dict[mut_name] = {'mutation': mut_name}

    results_dict[mut_name][f'ddG_BuildModel.{chain}'] = ddg

    print('-' * 40)

df = pd.DataFrame(results_dict.values())

Запуск BuildModel для мутации K1919N, цепь A
Результат не получен для мутации K1919N, цепь A
----------------------------------------
Запуск BuildModel для мутации K1919N, цепь B
Результат не получен для мутации K1919N, цепь B
----------------------------------------


In [ ]:
#Скачивание таблицы 

df_sorted = df.copy()
df_sorted["mutation_numeric"] = df_sorted["mutation"].str.extract(r'(\d+)').astype(int)
df_sorted = df_sorted.sort_values("mutation_numeric", ascending=False).drop(columns="mutation_numeric")

file_path = r" ##Путь до папки \benign_foldx.csv"
df_sorted.to_csv(file_path, index=False)